In [28]:
import numpy as np
from pydrake.all import (
    RigidTransform,
    RotationMatrix,
    StartMeshcat,
    RandomGenerator,
    Diagram,
)
import numpy as np

import sponana.utils
from sponana.planner.rrt import SpotProblem, rrt_planning
from sponana.planner.navigator import Navigator

import sponana.sim

In [29]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7009


In [30]:
meshcat.Delete()
meshcat.DeleteAddedControls()
rng = np.random.default_rng(145)  # this is for python
generator = RandomGenerator(rng.integers(0, 1000))  # this is for c++

add_spot = True
# simulation_time = -1  # run indefinitely until ESC is pressed
simulation_time = 10
debug = True
add_fixed_cameras = False
enable_arm_ik = True  # turn this off if you find the arm too annoying
use_teleop = False

simulator, diagram = sponana.sim.clutter_gen(
    meshcat,
    rng,
    debug=debug,
    simulation_time=simulation_time,
    add_spot=add_spot,
    add_fixed_cameras=add_fixed_cameras,
    enable_arm_ik=enable_arm_ik,
    use_teleop=use_teleop,
)

x_sample [-0.1   0.05]
y_sample [-2.00000000e-01  2.77555756e-17]
appended: x_points_append: [0.04999999999999999] y_points_append: [2.7755575615628914e-17]
dist 0.15
appended: x_points_append: [0.04999999999999999, -0.1] y_points_append: [2.7755575615628914e-17, 2.7755575615628914e-17]
dist 0.0
dist 0.2
dist 0.25
appended: x_points_append: [0.04999999999999999, -0.1, 0.04999999999999999] y_points_append: [2.7755575615628914e-17, 2.7755575615628914e-17, -0.19999999999999998]
x_sample [-0.1   0.05]
y_sample [-2.00000000e-01  2.77555756e-17]
appended: x_points_append: [-0.1] y_points_append: [-0.19999999999999998]
x_sample [-0.1   0.05]
y_sample [-2.00000000e-01  2.77555756e-17]
appended: x_points_append: [0.04999999999999999] y_points_append: [2.7755575615628914e-17]
dist 0.2
appended: x_points_append: [0.04999999999999999, 0.04999999999999999] y_points_append: [2.7755575615628914e-17, -0.19999999999999998]
dist 0.15
dist 0.25
appended: x_points_append: [0.04999999999999999, 0.049999999

==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html

==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html

==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html

==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The 

KeyboardInterrupt: 

In [ ]:
base_pose = np.array([1.00000000e00, 1.50392176e-12, 3.15001955e00])
q_start = base_pose
# q_goal = np.array([1.00000000e+00, 1.50392176e-12 -1, 3.15001955e+00])
# q_goal = np.array([1.00000000e+00, -0.5, 3.15001955e+00])
q_goal = np.array([-2, -2, 3.15001955e00])
# q_goal = np.array([0.20894849, -0.47792893, 0.2475])